# Gaussova eliminacija
---

## Općenito

Sustav $Ax=b$
se rješava u tri koraka (__bez pivotiranja__):

1. $A=LU$ (LU rastav, $O\big(\displaystyle\frac{2}{3}n^3\big)$ operacija),
2. $Ly=b$ (donje trokutrasti sustav, $n^2$ operacija),
3. $Ux=y$ (gornje torkutasti sustav, $n^2$ operacija).

Ukoliko pivotiramo, tada je

1. $PA=LU$, 
2. $Ly=P^T b$,
3. $Ux=y$. 

## LU rastav

In [1]:
function mylu{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    U=map(Float64,[zero(A[1,1]) for i=1:n, j=1:n]) # This acccepts blocks and numbers
    L=map(Float64,[zero(A[1,1]) for i=1:n, j=1:n])
    for k=1:n
        L[k,k]=one(A[1,1])
        for i=k+1:n
            L[i,k]=A[i,k]/A[k,k]
            for j=k+1:n
                A[i,j]=A[i,j]-L[i,k]*A[k,j]
            end
        end
        for j=k:n
            U[k,j]=A[k,j]
        end
    end
    L,U
end

mylu (generic function with 1 method)

In [2]:
A=rand(6,6)
# A=[2.0 2;3 4]

6×6 Array{Float64,2}:
 0.757035  0.26423   0.258824  0.444582  0.253857   0.725721 
 0.401192  0.596872  0.215737  0.461797  0.186466   0.492341 
 0.76668   0.115212  0.89138   0.25982   0.795885   0.654272 
 0.683031  0.700968  0.523178  0.429414  0.480718   0.676572 
 0.946519  0.136968  0.224276  0.55007   0.51893    0.091643 
 0.54143   0.459622  0.845367  0.563744  0.0815604  0.0321507

In [3]:
L,U=mylu(A)

(
[1.0 0.0 … 0.0 0.0; 0.529952 1.0 … 0.0 0.0; … ; 1.2503 -0.423337 … 1.0 0.0; 0.715199 0.592424 … -2.15358 1.0],

[0.757035 0.26423 … 0.253857 0.725721; 0.0 0.456843 … 0.0519339 0.107744; … ; 0.0 0.0 … 0.28954 -0.809514; 0.0 0.0 … 0.0 -2.34985])

In [4]:
L*U-A

6×6 Array{Float64,2}:
  0.0          0.0  0.0   0.0          0.0   0.0        
  0.0          0.0  0.0   0.0          0.0   0.0        
 -1.11022e-16  0.0  0.0   0.0          0.0   0.0        
  0.0          0.0  0.0  -1.11022e-16  0.0  -1.11022e-16
  0.0          0.0  0.0   0.0          0.0   0.0        
  0.0          0.0  0.0   0.0          0.0   2.22045e-16

## Trokutasti sustavi

In [5]:
function myU{T}(U::Array{T},b1::Array{T})
    b=deepcopy(b1)
    n=length(b)
    for i=n:-1:1
       for j=n:-1:i+1
            b[i]=b[i]-U[i,j]*b[j]
       end
        b[i]=b[i]/U[i,i]
    end
    b
end

function myL{T}(L::Array{T},b1::Array{T})
    b=deepcopy(b1)
    n=length(b)
    for i=1:n
        for j=1:i-1
            b[i]=b[i]-L[i,j]*b[j]
        end
        b[i]=b[i]/L[i,i]
    end
    b
end

myL (generic function with 1 method)

In [6]:
b=rand(6)

6-element Array{Float64,1}:
 0.992829
 0.265753
 0.705782
 0.748381
 0.301704
 0.307774

In [7]:
# Rijesimo sustav pomocu ugradjene funkcije:
x=A\b

6-element Array{Float64,1}:
  2.02928 
  0.339448
  0.291538
 -1.95671 
 -1.38635 
  0.707293

In [8]:
# Rijesimo sustav pomocu nasih funkcija
y=myL(L,b)

6-element Array{Float64,1}:
  0.992829
 -0.260399
 -0.386555
  0.240171
 -0.973968
 -1.66203 

In [9]:
x1=myU(U,y)

6-element Array{Float64,1}:
  2.02928 
  0.339448
  0.291538
 -1.95671 
 -1.38635 
  0.707293

In [10]:
# Usporedimo rjesenja
x-x1

6-element Array{Float64,1}:
  4.44089e-16
  2.22045e-16
 -5.55112e-17
 -2.22045e-16
 -4.44089e-16
  0.0        

## Brzina

Program `mylu()` je jako spor. Između ostalog, alocira nepotrebno tri matrice i ne računa s blok matricama.

Program se može reformulirati na načun da su i $L$ i $U$ spremljene u polje $A$, pri čemu se dijagonala od $L$ ne sprema, jer su svi elementi jednaki 1 (vidi [Introduction to Linear Algebra, str. 100][St09]):

[St09]: https://books.google.hr/books?id=M19gPgAACAAJ&dq=strang%20introduction&hl=hr&source=gbs_book_other_versions "Gilbert Strang, 'Introduction to Linear Algebra, 4th Edition', Wellesley-Cambridge Press, 2009"


In [13]:
function mylu1{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    for k=1:n-1
        rho=k+1:n
        A[rho,k]=A[rho,k]/A[k,k]
        A[rho,rho]=A[rho,rho]-A[rho,k]⋅A[k,rho]
    end
    A
end

mylu1 (generic function with 1 method)

In [14]:
mylu1(A)

6×6 Array{Float64,2}:
 0.757035   0.26423   0.258824    0.444582   0.253857   0.725721
 0.529952  -1.04283  -1.42397    -1.17791   -1.45324   -1.14736 
 1.01274    1.46188   5.78618     5.15462    5.69069    5.54907 
 0.902245   0.90018   0.936365  -10.0419    -9.99062   -9.79477 
 1.2503     1.44101   0.884707    0.987985   9.58192    9.15463 
 0.715199   1.13161   0.992048    0.986623   0.954355   0.358374

In [15]:
L,U

(
[1.0 0.0 … 0.0 0.0; 0.529952 1.0 … 0.0 0.0; … ; 1.2503 -0.423337 … 1.0 0.0; 0.715199 0.592424 … -2.15358 1.0],

[0.757035 0.26423 … 0.253857 0.725721; 0.0 0.456843 … 0.0519339 0.107744; … ; 0.0 0.0 … 0.28954 -0.809514; 0.0 0.0 … 0.0 -2.34985])

Usporedimo brzine LAPACK-ovog programa `lu()` i našeg naivnog programa `mylu()`na većoj dimenziji. 

Izvedite program par puta radi točnijeg mjerenja brzine.

In [18]:
n=512
A=rand(n,n)

512×512 Array{Float64,2}:
 0.0919826  0.292905    0.733022    …  0.64603     0.0549127  0.994975 
 0.857994   0.347594    0.967299       0.529304    0.735214   0.845291 
 0.667015   0.77826     0.670837       0.300541    0.990565   0.145595 
 0.64483    0.900804    0.169006       0.273833    0.575034   0.0933216
 0.451702   0.230523    0.00474206     0.139794    0.656611   0.449463 
 0.869137   0.585344    0.317795    …  0.244687    0.652848   0.874196 
 0.944877   0.698254    0.659188       0.792391    0.13794    0.0689758
 0.432484   0.390065    0.237245       0.692193    0.839645   0.487413 
 0.837263   0.949234    0.646632       0.0198505   0.400077   0.39341  
 0.653559   0.998439    0.00446579     0.163045    0.316857   0.972019 
 0.984158   0.878862    0.535231    …  0.130175    0.678641   0.179855 
 0.549629   0.418897    0.760914       0.860842    0.532577   0.0682183
 0.45418    0.887498    0.625917       0.815489    0.748958   0.768026 
 ⋮                                  ⋱ 

In [21]:
@time lu(A);

  0.047075 seconds (25 allocations: 6.009 MB)


In [22]:
@time mylu1(A);

  0.259748 seconds (10.14 k allocations: 687.223 MB, 8.11% gc time)


### Blok varijanta

`mylu()` je nekoliko desetaka puta sporiji od `lu()`. `mylu1()` je malo sporiji od `lu()`.

Probajmo s blokovima:

Preradimo `mylu1()` za rad s blokovima (nemamo ugrađeno pivotiranje!)

In [23]:
function mylu2{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    for k=1:n-1
        for rho=k+1:n
            A[rho,k]=A[rho,k]/A[k,k]
            for l=k+1:n
                A[rho,l]=A[rho,l]-A[rho,k]*A[k,l]
            end
        end
    end
    A
end

mylu2 (generic function with 1 method)

Napravimo prvo mali test:

In [24]:
# Probajte k,l=32,16 i k,l=64,8
k,l=4,4
Ab=[rand(k,k) for i=1:l, j=1:l]

4×4 Array{Array{Float64,2},2}:
 [0.947704 0.158683 0.0365133 0.430097; 0.428234 0.351874 0.269765 0.933251; 0.355974 0.0656793 0.0935246 0.527602; 0.438834 0.191424 0.816444 0.747859]  …  [0.0568117 0.882411 0.319672 0.67604; 0.900933 0.921685 0.489064 0.0914812; 0.909417 0.123832 0.160815 0.802153; 0.0409489 0.971473 0.215725 0.515026]  
 [0.659467 0.241069 0.631979 0.81001; 0.879621 0.807997 0.786473 0.880301; 0.192972 0.0446128 0.0623748 0.512638; 0.0579153 0.918366 0.144209 0.889487]      [0.143023 0.554266 0.578937 0.0400173; 0.914674 0.765108 0.766956 0.477176; 0.0185753 0.467021 0.28996 0.101813; 0.835942 0.447649 0.979307 0.726227]   
 [0.780562 0.161735 0.797171 0.91304; 0.89842 0.855968 0.432512 0.123992; 0.660866 0.325465 0.741242 0.749813; 0.548014 0.760475 0.898777 0.818984]          [0.453629 0.482695 0.480605 0.492865; 0.0743455 0.333364 0.60314 0.815718; 0.990425 0.964408 0.298062 0.801174; 0.334847 0.259367 0.395519 0.872074]    
 [0.783752 0.00306074 0.139413 0.479298

In [25]:
A0=mylu2(Ab)

4×4 Array{Array{Float64,2},2}:
 [0.947704 0.158683 0.0365133 0.430097; 0.428234 0.351874 0.269765 0.933251; 0.355974 0.0656793 0.0935246 0.527602; 0.438834 0.191424 0.816444 0.747859]   …  [0.0568117 0.882411 0.319672 0.67604; 0.900933 0.921685 0.489064 0.0914812; 0.909417 0.123832 0.160815 0.802153; 0.0409489 0.971473 0.215725 0.515026]   
 [0.294233 0.169451 -0.00419721 0.705395; 1.16477 2.16882 -4.05093 0.658628; -0.225043 0.0445881 1.16346 -0.0615455; 0.211502 3.56203 -4.00642 -0.550831]     [-0.0514242 -0.546302 0.25051 -0.534327; 2.55155 -2.39987 -0.156712 2.4016; -1.06436 0.540223 0.166268 -0.651702; 1.28085 -1.99081 -0.06724 3.75484]     
 [0.250836 -0.322055 0.688604 0.992707; 2.04311 2.51486 -6.35424 0.335318; 0.451977 0.428448 -0.910583 0.85042; 0.726875 2.12779 -3.96571 0.819561]           [0.531372 0.157745 -0.201386 0.271535; 1.96238 0.635309 -0.936208 1.47811; 1.72471 1.19023 -0.701353 0.335577; -0.150522 -0.587271 -0.309525 0.852971]   
 [0.546416 -0.566417 1.18458 0.19

In [26]:
# Provjera
U=triu(A0)
L=tril(A0)
for i=1:maximum(size(L))
    L[i,i]=eye(L[1,1])
end

In [27]:
Res=L*U-Ab

4×4 Array{Array{Float64,2},2}:
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]                                                                                                               …  [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]                           
 [0.0 2.77556e-17 1.11022e-16 -1.11022e-16; -1.11022e-16 2.22045e-16 0.0 1.11022e-16; 2.77556e-17 1.38778e-17 6.93889e-18 0.0; 1.11022e-16 -4.44089e-16 -2.77556e-16 -2.22045e-16]     [0.0 0.0 0.0 0.0; -2.22045e-16 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 2.22045e-16]          
 [0.0 2.77556e-17 1.11022e-16 0.0; -3.33067e-16 -1.11022e-16 -5.55112e-17 -2.22045e-16; 0.0 0.0 1.11022e-16 0.0; 0.0 1.11022e-16 0.0 1.11022e-16]                                      [0.0 0.0 5.55112e-17 0.0; 0.0 2.22045e-16 1.11022e-16 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]   
 [-1.11022e-16 4.16334e-17 5.55112e-17 0.0; 0.0 0.0 0.0 5.55112e-17; -4.44089e-16 -2.22045e-16 -1.11022e-16 3.33067e-16; 5.55112e-17

In [28]:
# pretvaranje blok matrice u obicnu
unblock(A) = mapreduce(identity, hcat, [mapreduce(identity, vcat, A[:,i]) for i = 1:size(A,2)])

unblock (generic function with 1 method)

In [29]:
norm(unblock(Res))

1.490802012417364e-15

Sada probajmo veću dimenziju:

In [33]:
# Probajmo vece dimenzije (n=k*l)
k,l=32,16
Ab=[rand(k,k) for i=1:l, j=1:l];

In [34]:
@time mylu2(Ab);

  0.091593 seconds (7.29 k allocations: 26.653 MB, 7.56% gc time)


Vidimo da je `mylu2()` gotovo jednako brz kao `lu()`, uz napomenu da `mylu2()` nema ugrađeno pivotiranje. 

## Pivotiranje

Stanardne implementacije uvijek računaju Gaussovu eliminaciju s _parcijalnim pivotiranjem_:

> u svakom koraku se retci pivotiranju tako da pivotni element ima najveću apsolutnu vrijednst u danom stupcu. Na taj 
> način je 
> 
> $$|L_{ij}| \leq 1,$$
> 
> što u praksi dovoljno spriječava rast elemenata.


In [35]:
A=rand(5,5)
L,U,P=lu(A)

(
[1.0 0.0 … 0.0 0.0; 0.521994 1.0 … 0.0 0.0; … ; 0.506594 0.742406 … 1.0 0.0; 0.479534 0.394967 … 0.116674 1.0],

[0.824772 0.304595 … 0.887334 0.202908; 0.0 0.500854 … -0.0179088 0.501239; … ; 0.0 0.0 … -0.449324 -0.171969; 0.0 0.0 … 0.0 -0.344188],

[5,4,3,2,1])

In [36]:
L*U-A[P,:]

5×5 Array{Float64,2}:
  0.0          0.0  0.0  0.0  0.0
 -5.55112e-17  0.0  0.0  0.0  0.0
 -2.77556e-17  0.0  0.0  0.0  0.0
  0.0          0.0  0.0  0.0  0.0
 -5.55112e-17  0.0  0.0  0.0  0.0

### Potpuno pivotiranje

Sljedeći program računa Gaussovu eliminaciju s _potpunim pivotiranjem_ - u svakom koraku retci i stupci zamijene na način da se na pivotnu poziciju dovede element koji ima najveću apsolutnu vrijednost u trenutnoj podmatrici.

In [44]:
function gecp{T}(A1::Array{T})
    # Gaussova eliminacija s potpunim pivotiranjem
    # Izlaz: Pr*L*U*Pc'=A ili Pr'*A*Pc=L*U
    A=deepcopy(A1)
    n,m=size(A)
    Pr=eye(n,n)
    Pc=eye(n,n)
    D=zeros(n)
    for i=1:n-1
        am1,im1=findmax(abs(A[i:n,i:n]),1)
        am,JJ=findmax(am1)
        II=mod(im1[JJ],(n-i+1))
        if II==0
            II=n-i+1
        end
        imax=II+i-1
        jmax=JJ+i-1
        #  zamijena redaka
        if (imax != i)
            temp = Pr[:,i]
            Pr[:,i] = Pr[:,imax]
            Pr[:,imax] = temp
            temp = A[i,:]
            A[i,:] = A[imax,:]
            A[imax,:] = temp
        end
        # zamijena stupaca
        if (jmax != i)
            temp = Pc[:,i]
            Pc[:,i] = Pc[:,jmax]
            Pc[:,jmax] = temp
            temp = A[:,i]
            A[:,i] = A[:,jmax]
            A[:,jmax] = temp
        end
        # eliminacija
        D[i]=A[i,i]
        A[i+1:n,i] = A[i+1:n,i]/D[i]
        A[i+1:n,i+1:n] = A[i+1:n,i+1:n] - A[i+1:n,i]*A[i,i+1:n]'
        A[i,i+1:n]=A[i,i+1:n]/D[i]
    end
    D[n]=A[n,n]
    L=eye(n,n)+tril(A,-1)
    U=eye(n,n)+triu(A,1)
    U=diagm(D)*U
    L,U,Pr,Pc
end

gecp (generic function with 1 method)

In [45]:
n=5
A=rand(n,n)
b=rand(n)

5-element Array{Float64,1}:
 0.294559
 0.351749
 0.69913 
 0.109168
 0.419442

In [46]:
L,U,Pr,Pc=gecp(A)

(
[1.0 0.0 … 0.0 0.0; 0.383386 1.0 … 0.0 0.0; … ; 0.64982 0.333702 … 1.0 0.0; 0.99698 0.251847 … 0.940898 1.0],

[0.983242 0.204353 … 0.687532 0.167024; 0.0 0.806194 … 0.309776 0.476813; … ; 0.0 0.0 … -0.230477 -0.167206; 0.0 0.0 … 0.0 0.221589],

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 1.0; … ; 1.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.0 0.0],

[0.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 1.0 0.0 … 0.0 0.0])

In [47]:
Pr*L*U*Pc'-A

5×5 Array{Float64,2}:
 0.0  0.0  0.0           0.0          0.0
 0.0  0.0  0.0           0.0          0.0
 0.0  0.0  1.11022e-16   0.0          0.0
 0.0  0.0  0.0           0.0          0.0
 0.0  0.0  2.77556e-17  -5.55112e-17  0.0

In [48]:
y=myL(L,Pr'*b)

5-element Array{Float64,1}:
  0.109168
  0.657277
 -0.183956
  0.218331
 -0.308184

In [49]:
z=myU(U,y)

5-element Array{Float64,1}:
  0.125303 
  1.68781  
 -0.375156 
  0.0616839
 -1.39079  

In [50]:
x=Pc*z

5-element Array{Float64,1}:
  1.68781  
 -0.375156 
 -1.39079  
  0.0616839
  0.125303 

In [51]:
A*x-b

5-element Array{Float64,1}:
 -1.11022e-16
  1.11022e-16
 -1.11022e-16
  5.55112e-17
 -5.55112e-17

## Točnost

Neka je zadan sustav $Ax=b$, pri čemu je matrica $A$ regularna.

Da bi primijenili koncepte iz bilježnice [NA04_Pogreska_unatrag_i stabilni_algoritmi](NA04_Pogreska_unatrag_i_stabilni_algoritmi.ipynb), potrebno je:

1. napraviti teoriju smetnje za dani problem
2. analizirati pogreške algoritma (Gaussove eliminacije)

### Teorija smetnje

Neka je 

$$
(A+\delta A)\hat x=(b+\delta b)
$$

za neki $\hat x=x+\delta x$.

Želimo ocijeniti 

$$
\frac{\| \hat x - x \|}{\| x\|} \equiv \frac{\| \delta x\|}{\| x\|}.
$$

Uvedimo oznake (npr. prema [Matrix Computations, poglavlje 2.6.2][GVL13])

$$
\delta A=\varepsilon F, \quad \delta b=\varepsilon f, \qquad \hat x=x(\varepsilon),
$$
čime smo dobili jednodimenzionalni problem 

$$
(A+\varepsilon F)x(\varepsilon)=b+\varepsilon f.
$$

za neke (nepoznate) matricu $F$ i vektor $f$. 

Deriviranje po $\varepsilon$ daje

$$
Fx(\varepsilon)+(A+\varepsilon F)\dot x(\varepsilon)=f.
$$

Uvrštavanje $\varepsilon=0$ daje

$$
F x+A\dot x(0)=f,
$$

odnosno

$$
\dot x(0)=A^{-1}(f-Fx).
$$

Taylorov razvoj oko $x(0)$ glasi

$$
x(\varepsilon)=x(0)+\varepsilon \dot x(0) +O(\varepsilon^2),
$$

odnosno, uz zanemarivanje člana $O(\varepsilon^2)$,

$$
\hat x-x=\varepsilon A^{-1}(f-Fx)=A^{-1} (\varepsilon f + \varepsilon F x) = A^{-1} (\delta b + \delta A x).
$$

Svojstva norme povlače

$$
\| \hat x-x\|\leq \| A^{-1} \| (\| \delta b \|  + \| \delta A \| \cdot \|  x\| ).
$$

Konačno, zbog $\| b\| \leq \| A\| \| x\|$, imamo

$$
\frac{\| \hat x-x\|}{\| x\|}\leq \| A\|  \cdot \| A^{-1} \| \bigg(\frac{\| \delta b \|}{\|b\|}  + \frac{\| \delta A \|}{ \|  A\|} \bigg). \tag{1}
$$

Broj 
$$
\kappa(A)\equiv \| A\|  \cdot \| A^{-1} \|
$$ 

je __uvjetovanost__ (__kondicija__)  matrice $A$ i kazuje nam 

> koliko se relativno uvećaju relativne promjene u polaznim podacima (matrici $A$ i vektoru $b$).

Pogledajmo primjer iz [Numeričke matematike, str. 42][RS04]:


[GVL13]: https://books.google.hr/books?id=X5YfsuCWpxMC&printsec=frontcover&hl=hr#v=onepage&q&f=false "G. Golub and C. F Van Loan, 'Matrix Computations', 4th Edition, John Hopkins, Baltimore, 2013" 

[RS04]: http://www.mathos.unios.hr/pim/Materijali/Num.pdf "R. Scitovski, 'Numerička matematika', Sveučilište u Osijeku, Osijek, 2004."

In [52]:
A= [0.234 0.458; 0.383 0.750]

2×2 Array{Float64,2}:
 0.234  0.458
 0.383  0.75 

In [53]:
b=[0.224;0.367]

2-element Array{Float64,1}:
 0.224
 0.367

In [54]:
x=A\b

2-element Array{Float64,1}:
 -1.0
  1.0

In [55]:
δb=[0.00009; 0.000005]
x1=A\(b+δb)

2-element Array{Float64,1}:
 -0.241744
  0.612791

In [56]:
cond(A), norm(δb)/norm(b), norm(x1-x)/norm(x)

(11322.197586092605,0.0002096449170953002,0.6020311134825742)

In [57]:
δA=[-0.001 0;0 0]
x2=(A+δA)\b

2-element Array{Float64,1}:
 0.129518
 0.423193

In [58]:
cond(A), norm(δA)/norm(A), norm(x2-x)/norm(x)

(11322.197586092605,0.0010134105230118603,0.896804787832142)

### Pogreška Gaussove eliminacije

Prema [Matrix Computations, poglavlje 3.3][GVL13], za izračunate faktore
$\hat L$ i $\hat U$ vrijedi

$$
\hat L\cdot \hat U = A+\delta A
$$

gdje je (nejednakost se čita po elementima matrica, $\varepsilon$ je sada točnost stroja)

$$
| \delta A|\leq 3(n-1) \varepsilon (|A|+|\hat L| \cdot |\hat U|) +O(\varepsilon^2).
$$

Zanemarivanje člana $O(\varepsilon^2)$ i prelazak na normu daju

$$
\|\delta A \| \approx \leq  O(n)\varepsilon (\| A\| + \| \hat L\| \cdot \| \hat U\|),
$$

pa je 

$$
 \frac{\|\delta A \|}{\|A\|} \leq O(n)\varepsilon \bigg(1+\frac{\| \hat L\| \cdot \| \hat U\|}{\|A\|}\bigg).
$$

Ukoliko se Gaussova eliminacija radi s pivotiranjem, tada će najvjerojatnije zadnji kvocijent također biti malen 
($\approx 1$). Također, pogreška kod rješavanja trokutastih sustava nije veća od navedene pa, uvrštavanjme u (1), slijedi 
da za relativnu pogrešku izračunatog rješenja vrijedi

$$
\frac{\| \hat x-x\|}{\| x\|}\leq \kappa(A) O(n\varepsilon).
$$

> __Ukoliko je kondicija matrice velika, rješenje može biti netočno.__

[GVL13]: https://books.google.hr/books?id=X5YfsuCWpxMC&printsec=frontcover&hl=hr#v=onepage&q&f=false "G. Golub and C. F Van Loan, 'Matrix Computations', 4th Edition, John Hopkins, Baltimore, 2013" 

In [59]:
n=10
v=rand(n)

10-element Array{Float64,1}:
 0.685816 
 0.0897764
 0.331064 
 0.0372276
 0.589024 
 0.472516 
 0.715734 
 0.379478 
 0.211293 
 0.910941 

In [60]:
# Vandermonmde-oove matrice imaju veliku kondiciju.
A=Array(Float64,n,n)
for i=1:n
    A[:,i]=v.^(i-1)
end
A=A'

10×10 Array{Float64,2}:
 1.0        1.0          1.0          …  1.0          1.0         1.0     
 0.685816   0.0897764    0.331064        0.379478     0.211293    0.910941
 0.470343   0.00805981   0.109603        0.144004     0.0446448   0.829814
 0.322569   0.000723581  0.0362857       0.0546462    0.00943314  0.755912
 0.221223   6.49605e-5   0.0120129       0.020737     0.00199316  0.688591
 0.151718   5.83193e-6   0.00397703   …  0.00786925   0.00042114  0.627266
 0.104051   5.2357e-7    0.00131665      0.00298621   8.8984e-5   0.571403
 0.0713596  4.70042e-8   0.000435895     0.0011332    1.88017e-5  0.520514
 0.0489396  4.21987e-9   0.000144309     0.000430025  3.97267e-6  0.474158
 0.0335635  3.78845e-10  4.77755e-5      0.000163185  8.39399e-7  0.43193 

In [61]:
b=rand(n)

10-element Array{Float64,1}:
 0.334644 
 0.0811179
 0.589614 
 0.187764 
 0.203573 
 0.28884  
 0.697674 
 0.708069 
 0.947738 
 0.980163 

In [62]:
x=A\b

10-element Array{Float64,1}:
     -4.79216e5
 -80610.2      
     -1.44658e6
  26033.6      
      5.48085e5
     -1.01539e6
      2.61181e5
      1.91869e6
      2.68998e5
  -1200.16     

In [63]:
cond(A)

6.923611014281535e7

In [64]:
Ab=Array(BigFloat,n,n)
bb=Array(BigFloat,n)
for i=1:n
    for j=1:n
        Ab[i,j]=convert(BigFloat,A[i,j])
    end
    bb[i]=convert(BigFloat,b[i])
end
xb=Ab\bb

10-element Array{BigFloat,1}:
 -4.792158503668085782666477516051190605395196444863196051513226993039490422534515e+05
 -8.061018496756140623567151989892980328620704884736550927917870865139192526945689e+04
 -1.446575492464216620006139152470522853113916314459885974120964336074577165601753e+06
  2.60336092314774617703561270785112543866059724626562942793465087499468563492546e+04 
  5.480846585606001585596888532808330745481192808726843282271922504309250616374176e+05
 -1.015388838504983553890382090225869901811778368634711862024030634880889541411244e+06
  2.611813513018825426000281531077747947724734807897540650625343754299238967605611e+05
  1.918692928913689090127992433184809986005057929766108990895560669016197026205215e+06
  2.689983089593238674556097236794854235412431694707032140113175080617422960595239e+05
 -1.200156019719334741434207459004556398892564828521402837954932777927462476072208e+03

In [65]:
norm(xb-x)/norm(xb)

1.290039177873875592197949019130908033581575644778761003122730583348990459824275e-11

### Umjetno loša kondicija

In [66]:
A=[1 1; 1 2]
b=[1;3]
x=A\b
@show x,cond(A)
A1=[1e-4 1e-4;1 2]
b1=[1e-4;3]
x1=A1\b1
x,cond(A1),x-x1

(x,cond(A)) = ([-1.0,2.0],6.854101966249685)


([-1.0,2.0],50000.00017991671,[8.88178e-16,-4.44089e-16])

## Rezidual


Izračunato rješenje $\hat x$ sustava $Ax=b$ je točno rješenje nekog sličnog sustava (vidi [Afternotes on Numerical Analysis, str. 128][Ste96]):


$$ 
(A+\delta A)\hat x=b. \tag{1}
$$

__Rezidual__ (ili __ostatak__) definiramo kao 

$$
r=b-A\hat x.
$$

Tada je 

$$
0=b-(A+\delta A)\hat x=r- \delta A\hat x
$$

pa je 

$$ 
\| r\| \leq \| \delta A\hat x \| \leq \| \delta A\| \cdot \|\hat x \|,
$$

odnosno

$$
\frac{\|  \delta A\|}{\|A \|} \geq \frac{\|r\|}{\| A\| \cdot \|\hat x \|}.
$$

Dakle,

> ako  _relativni rezidual_ 
>
> $$ \frac{r}{\| A\| \cdot \|\hat x \|}$$
> 
> ima veliku normu, tada __rješenje nije izračunato stabilno.__

S druge strane, ako relativni rezidual ima malu normu, tada je rješenje izračunato stabilno. Naime, za

$$
\delta A=\frac{r\hat x^T}{\|\hat x\|^2}
$$

vrijedi (1):

$$
b-(A+\delta A)\hat x=(b-A\hat x)-\delta A \hat x = r-\frac{r\hat x^T \hat x}{\|\hat x\|^2}
= r-\frac{r \|\hat x^T \hat x\|}{\|\hat x\|^2}=r-r=0.
$$

Također vrijedi

$$
\frac{\|  \delta A\|}{\|A \|}  \leq  \frac{\|r\|\|\hat x \|}{\| A\| \cdot \|\hat x \|^2}=
\frac{\|r\|}{\| A\| \cdot \|\hat x \|}.
$$




[Ste96]: https://books.google.hr/books?id=w-2PWh01kWcC&printsec=frontcover&hl=hr#v=onepage&q&f=false    "G. W. Stewart, 'Afternotes on Numerical Analysis', SIAM, Philadelphia, 1996"

In [67]:
r=b-A*x

2-element Array{Float64,1}:
 0.0
 0.0

In [68]:
norm(r)/(norm(A)*norm(x))

0.0